In [20]:
import pandas as pd
import yake
import pandas as pd
import numpy as np
import re
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
import tldextract
import pycountry
import pycountry_convert as pc
import ast
from utils import *

In [21]:
data_spacy = pd.read_csv("/content/rule_based_nat_cat_4.csv")
print(data_spacy.shape)
data = data_spacy.copy()

(20599, 17)


In [22]:
# YAKE keyword extractor
kw_extractor = yake.KeywordExtractor(
    lan="en",       # Language
    n=1,            # Max number of words per keyword (can set to 2 or 3 if you want phrases)
    dedupLim=0.9,   # How similar keywords can be before deduplication
    top=5,          # How many keywords to extract
    features=None   # Use default features
)

# Extract keywords from each title
def extract_keywords(text):
    keywords = kw_extractor.extract_keywords(text)
    return [kw for kw, score in keywords]

# Apply it
data['keywords'] = data['clean_title'].apply(extract_keywords)



In [23]:
# Feature engineering for date column. Will be adding month and season features
data['seendate'] = pd.to_datetime(data['seendate'], errors='coerce')
data['month'] = data['seendate'].dt.month
data['season'] = data['month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Autumn', 10: 'Autumn', 11: 'Autumn'
})

In [24]:
# Feature engineering for locations columns. Imported these functions from utils file
data['locations_string'] = data['locations'].apply(location_list_to_string)
data['country_extracted'] = data['locations_string'].apply(get_country_from_text)
data['continent'] = data['country_extracted'].apply(get_continent)

In [25]:
data.head()

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry,subdomain,domain_root,...,locations,has_location,has_natcat_keyword,matched_keyword,keywords,month,season,locations_string,country_extracted,continent
0,https://www.ctvnews.ca/climate-and-environment...,NaN,Iceland volcanoes bring tourists to island cou...,2024-01-01 22:30:00,https://www.ctvnews.ca/content/dam/ctvnews/en/...,ctvnews.ca,English,Canada,NaN,ctvnews,...,['Iceland'],True,True,volcano,"[Iceland, country, volcanoes, bring, tourists]",1,Winter,Iceland,Iceland,Europe
1,https://news.yahoo.com/tornados-scorchers-ice-...,NaN,"Tornados , scorchers and ice storm : Top 10 we...",2024-01-01 13:15:00,https://s.yimg.com/ny/api/res/1.2/PXdWVXp40q9s...,news.yahoo.com,English,United States,news,yahoo,...,"['Abilene', 'San Angelo']",True,True,storm,"[Top, Tornados, Abilene, San, Angelo]",1,Winter,Abilene San Angelo,None,Unknown
2,https://globalnews.ca/news/10198334/japan-eart...,https://globalnews.ca/news/10198334/japan-eart...,Japan earthquakes : Coastal residents told to ...,2024-01-01 16:15:00,https://globalnews.ca/wp-content/uploads/2024/...,globalnews.ca,English,Canada,NaN,globalnews,...,['Japan'],True,True,earthquake,"[National, Coastal, Japan, earthquakes, warnings]",1,Winter,Japan,Japan,Asia
3,https://reliefweb.int/report/philippines/dswd-...,NaN,DSWD DROMIC Report on the Tornado Incident in ...,2024-01-01 09:30:00,https://reliefweb.int/sites/default/files/styl...,reliefweb.int,English,Syria,NaN,reliefweb,...,"['Brgy', 'Anao']",True,True,tornado,"[Rizal, Anao, Tarlac, December, Philippines]",1,Winter,Brgy Anao,None,Unknown
4,https://www.columbian.com/news/2023/dec/31/new...,NaN,New state report : Wildfire smoke increased de...,2024-01-01 05:30:00,https://pcdn.columbian.com/wp-content/themes/c...,columbian.com,English,United States,NaN,columbian,...,"['Spokane', 'Washington']",True,True,wildfire,"[Wildfire, Spokane, Washington, report, state]",1,Winter,Spokane Washington,None,Unknown


In [26]:
selected_cols = [
    'matched_keyword', 'keywords',
    'month', 'season', 'country_extracted']
df_cluster = data[selected_cols].copy()
df_cluster.head()

,matched_keyword,keywords,month,season,country_extracted
0,volcano,"[Iceland, country, volcanoes, bring, tourists]",1,Winter,Iceland
1,storm,"[Top, Tornados, Abilene, San, Angelo]",1,Winter,None
2,earthquake,"[National, Coastal, Japan, earthquakes, warnings]",1,Winter,Japan
3,tornado,"[Rizal, Anao, Tarlac, December, Philippines]",1,Winter,None
4,wildfire,"[Wildfire, Spokane, Washington, report, state]",1,Winter,None


In [9]:
df_cluster['matched_keyword'].value_counts()

,count
matched_keyword,
earthquake,5664
flood,4257
tornado,3240
wildfire,3237
storm,2449
volcano,910
hurricane,369
tsunami,223
eruption,195


In [27]:
# Keywords to drop
drop_keywords = ['landslide', 'mudslide', 'hurricane', 'tsunami', 'storm']
# Drop unwanted rows
df_cluster = df_cluster[~df_cluster['matched_keyword'].isin(drop_keywords)]
# Replace 'eruption' with 'volcano'
df_cluster['matched_keyword'] = df_cluster['matched_keyword'].replace('eruption', 'volcano')

In [28]:
df_cluster['matched_keyword'].value_counts()

,count
matched_keyword,
earthquake,5664
flood,4257
tornado,3240
wildfire,3237
volcano,1105


In [12]:
df_cluster['month'].value_counts()

,count
month,
6,1652
5,1617
7,1579
8,1546
11,1512
9,1500
4,1414
12,1397
1,1388


In [13]:
df_cluster['country_extracted'].value_counts()

,count
country_extracted,
United States,791
Japan,556
Indonesia,542
Iceland,348
Canada,240
...,...
Türkiye,1
Guadeloupe,1
Rwanda,1


In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sentence_transformers import SentenceTransformer


# Fill missing countries with 'Unknown'
df_cluster['country_extracted'] = df_cluster['country_extracted'].fillna('Unknown')

# Convert list of keywords to a string
df_cluster['keywords_str'] = df_cluster['keywords'].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
# Load SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")
keyword_embeddings = model.encode(df_cluster['keywords_str'].tolist(), show_progress_bar=True)

# Preprocessing Tabular Data
features = ['matched_keyword', 'month', 'season', 'country_extracted']
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['matched_keyword', 'season', 'country_extracted']),
    ('num', StandardScaler(), ['month'])
])

X_tabular = preprocessor.fit_transform(df_cluster[features])

# Combine Features
X_combined = np.hstack([X_tabular, keyword_embeddings])

# Define Clustering Algorithms
models = {
    'KMeans': KMeans(n_clusters=5, random_state=42, n_init=10),
    'Agglomerative': AgglomerativeClustering(n_clusters=5),
    'DBSCAN': DBSCAN(eps=1.5, min_samples=5)  # You can tune this
}


# Evaluation Function
def evaluate_model(name, model, X):
    try:
        labels = model.fit_predict(X)
        unique_labels = set(labels)
        if len(unique_labels) < 2 or len(unique_labels) == len(X):
            print(f"\n{name}: Not enough clusters formed for meaningful evaluation.")
            return
        sil = silhouette_score(X, labels)
        cal = calinski_harabasz_score(X, labels)
        db = davies_bouldin_score(X, labels)

        print(f"\n{name} Clustering Evaluation:")
        print(f"  Silhouette Score:        {sil:.4f}")
        print(f"  Calinski-Harabasz Index: {cal:.2f}")
        print(f"  Davies-Bouldin Index:    {db:.4f}")
    except Exception as e:
        print(f"{name} failed: {e}")

# Run and Evaluate All Models
for name, model in models.items():
    evaluate_model(name, model, X_combined)


Batches:   0%|          | 0/547 [00:00<?, ?it/s]


KMeans Clustering Evaluation:
  Silhouette Score:        0.1940
  Calinski-Harabasz Index: 3558.23
  Davies-Bouldin Index:    1.9314

Agglomerative Clustering Evaluation:
  Silhouette Score:        0.1975
  Calinski-Harabasz Index: 3375.53
  Davies-Bouldin Index:    1.8439

DBSCAN Clustering Evaluation:
  Silhouette Score:        -0.1519
  Calinski-Harabasz Index: 48.10
  Davies-Bouldin Index:    1.5355


## Summary: Comparing Transformer vs. Spacy for Disaster-Relevant Title Extraction
In comparing disaster-related titles extracted by transformer and Spacy-based models, the transformer-based approach provided significantly more accurate and context-rich natural catastrophe events. Titles like “Massive earthquake hits Japan, horrifying visuals surface” and “New Year Day flood alerts issued in Cambridgeshire” are timely, specific, and aligned with real-world disaster triggers. In contrast, the Spacy model included non-disaster content such as “Ireland is surrounded by volcanoes” or “Iceland volcanoes bring tourists to island country”, which are geologically factual but not catastrophic.

Importantly, the initial filtered dataset size was larger in the transformer notebook (23,634 rows) compared to only 20,599 rows in the Spacy dataset, even before any further balancing or cleanup. This suggests that the transformer-based pipeline captured more disaster-relevant content up front, providing a stronger foundation for downstream clustering and analysis.

The transformer model excelled by embedding the full sentence context, capturing urgency and consequence, while the Spacy pipeline relied more on surface-level keyword detection. This difference in semantic depth explains why transformer-based inputs lead to much stronger clustering performance and higher silhouette scores (aproximately 0.85) compared to the Spacy model (0.13 - 0.19).